In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

## 示例选择器生成精准提示词

1. 准备示例
2. 计算示例的语义相似度
3. 使用 FewShotPromptTemplate，根据 2. 获得的相似度获取 prompt
4. 根据 prompt 获取结果

In [1]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.llms import OpenAI

In [24]:
example_prompt = PromptTemplate(
    template="示例输入:{input}\n示例输出:{output}",
    input_variables=["input", "output"]
)

examples = [
    {"input":"海盗", "output":"船"},
    {"input":"飞行员", "output":"飞机"},
    {"input":"驾驶员", "output":"汽车"},
    {"input":"树", "output":"地面"},
    {"input":"鸟", "output":"鸟巢"},
]

In [25]:
! pip install -q tiktoken faiss-cpu

In [26]:
# 计算提供的示例的语义相似度
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # List of examples to use in the prompt
    examples,
    
    # An initialized embedding API interface, 用于测量语义相似性
    OpenAIEmbeddings(),
    
    # A vector store DB interface class, 用于存储嵌入和进行相似性搜索的 VectorStore 类
    FAISS,
    
    # 生成的示例数
    k=2
)

In [27]:
# 【few shot 提示模板】生成器
similar_prompt = FewShotPromptTemplate(
    # 示例集合，其中的示例将会被选择
    example_selector=example_selector,
    
    # 提示词
    example_prompt=example_prompt,
    
    # 自定义项，将分别添加到 提示顶部 和 提示底部
    prefix="根据下面示例写出输出",
    suffix="输入:{noun}\n输出:",
    
    # 输入变量名
    input_variables=["noun"]
)

In [32]:
# 获取提示
final_prompt = similar_prompt.format(noun='苹果')
print(final_prompt)

根据下面示例写出输出

示例输入:鸟
示例输出:鸟巢

示例输入:树
示例输出:地面

输入:苹果
输出:


In [33]:
llm = OpenAI()

In [34]:
llm(final_prompt)

'树上'